### Nuißl Sandra, 14.08.2023
## "Empirische Evaluation von ‚State Of The Art‘ Topic Modeling Ansätze am Beispiel von Produktreviews für die Entscheidungsunterstützung in Unternehmen"
### - Data Analyse und Data Preperation -
<center><img src="Titelbild.jpg" height="300px" width="1100px"/></center>
<center><font size="1">https://www.cloudways.com/blog/wp-content/uploads/Product-Review-1024x576.jpg</font></center>


<hr>

## **Inhaltsverzeichnis**

<ul>1. Aufbau des Jupyter Notebooks</ul>
<ul>2. Instalation und Imports</ul>
    <ul>
     <ul>2.1. Installationen</ul>
     <ul>2.2. Imports</ul>
    </ul>
<ul>3. Laden der Amazon Daten</ul>
    <ul>
     <ul>3.1. Entpacken der Zip Files</ul>
     <ul>3.2. Laden des Datensatzes in einen Data Frame</ul>
    </ul>
<ul>4. Data Preperation</ul>
    <ul>
     <ul>4.1. Lowercasing</ul>
     <ul>4.2. Lemmatisierung</ul>
     <ul>4.3. Stemming</ul>
     <ul>4.4. Entfernung von Satzzeichen</ul>
     <ul>4.5. Entfernung von Stopwords</ul>
     <ul>4.6. Entfernung von Zahlen</ul>
     <ul>4.7. Entfernung von nicht ASCII konformen Wörtern</ul>
    </ul>
<ul>5. Literaturverzeichnis</ul>
<hr>

## 1. Aufbau des Jupyter Notebooks
Dieses Jupyter Notebook ist Teil der Master Thesis "Empirische Evaluation von ‚State Of The Art‘ Topic Modeling Ansätze am Beispiel von Produktreviews für die Entscheidungsunterstützung in Unternehmen".
Es setzt sich zusammen aus der Datenanalyse der Amazon Daten, sowie der Data Preperation. Die dient als Vorbereitung für die spätere Implementierung und Evaluierung der Topic Modelling Modelle.

Die theoretischen Inhalte zu diesem Notebook sind der schriftlichen Ausarbeitung dieser Masterarbeit zu entnehmen.

## 2. Instalationen und Imports
### 2.1. Installationen

In [ ]:
# Instalationen
#% pip install gzip
#% pip install shutil
#% pip install nltk
#%pip install LanguageIdentifier

### 2.2. Imports

In [1]:
# Imports und Initalisierungen
import datetime
import random
import numpy as np
import pandas as pd
from pandas import json_normalize

# Progressbar
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from tqdm.auto import tqdm
tqdm_notebook.pandas()

# Entpacken der Files
import gzip
import shutil

# Beschleunigung
import json
import joblib
from joblib import Parallel, delayed


# Data Preprocessing
from langid.langid import LanguageIdentifier, model        # Ermittlung der Sprache von Texten
from LanguageIdentifier import predict

# Text
import re
import string
from string import punctuation

# Natural Language Processing
import nltk
from nltk.stem.wordnet import WordNetLemmatizer             # Lemmatisierung zur Textdimensionsreduktion
#nltk.download('averaged_perceptron_tagger')                 # Für POS-Tagging
#from nltk import word_tokenize                              # Tokenisierung
from nltk import pos_tag                                    # Bestimmung der grammatikalischen Token
from nltk.stem.snowball import SnowballStemmer              # Stemmer zur Textdimensionsreduktion
from nltk.corpus import stopwords                           # Zur entfernung der Stopwords
stopwords.words("english")                                  # Festlegung der Stoppwords
#nltk.download('punkt')                                      # Für den Export in ein Parquet File
#nltk.download('wordnet')                                    # Für den Export in ein Parquet File
nltk.download('stopwords')                                  # Herunterladen der Liste mit Stopwords


# Datenvisualisierung
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nuiss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3. Laden der Amazon Daten

### 3.1. Entzippen der Json Files
Die Amazon Datensätze sind aufgrund der großen Datenmengen als Zip Dateien gespeichert. Um diese in das Jupyter Notebook einlesen zu können, müssen daher die JSON Files zuerst entpackt werden.

Zum Entpacken der Files wird im Folgenden "gzip" in Verbindung mit "shutil" verwendet. 
Gzip ist ein Programm zur Daten Kompression (Free Software Foundation, 2022) während shutil ein Modul ist, welches diverse High-Level-Operationen zur Unterstützung beim Kopieren und Löschen von Dateien bietet (Python-Software-Foundation, 2023). 
Durch deren Kombination werden zuerst zwei Files geöffnet. file_in beschreibt hier das gezippte JSON file und bei file_out handelt es sich um ein leeres JSON File, in welches die Daten aus file_in mithilfe der Funktion "copyfileobj" von shutil kopiert werden (Erick, 2018). 

In [ ]:
# Entpacken der Datei und speichern in einem JSON File
# Quelle: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
# Review Daten
#with gzip.open('Sports_and_Outdoors_5.json.gz', 'rb') as file_in:
#    with open('Sports_and_Outdoors_5.json', 'wb') as file_out:
#        shutil.copyfileobj(file_in, file_out)

# Meta Daten
#with gzip.open('meta_Sports_and_Outdoors.json.gz', 'rb') as file_in:
#    with open('meta_Sports_and_Outdoors.json', 'wb') as file_out:
#        shutil.copyfileobj(file_in, file_out)

### 3.2. Laden des Datensatzes in einen Data Frame

Nachdem die Datensätze entpackt wurden, können diese geladen in einen Data Frame gespeichert werden.

In [3]:
# Review Daten
# Öffnen der Datei und speichern als Objekt in der Variable f
r = open('Sports_and_Outdoors_5.json')

# Laden des JSON Files in das Jupyter Notebook mit Beschleunigung
dataset_review = Parallel(n_jobs=-1)(delayed(json.loads)(line) for line in tqdm(r, desc ="Status der geladenen Zeilen: ", total = 2839940))

# Normalisieren der Daten und Laden in einen Data Frame
review_df = json_normalize(dataset_review)

# Ausgabe
#pd.concat([review_df], axis=1, sort = False, keys = ["Review Datensatz: Sports and Outdoors"])

Status der geladenen Zeilen:   0%|          | 0/2839940 [00:00<?, ?it/s]

In [2]:
# Meta Daten
# Öffnen der Datei und speichern als Objekt in der Variable f
m = open('meta_Sports_and_Outdoors.json')

# Laden des JSON Files in das Jupyter Notebook mit Beschleunigung
dataset_meta = Parallel(n_jobs=-1)(delayed(json.loads)(line) for line in tqdm(m, desc ="Status der geladenen Zeilen: ", total = 962300))

# Normalisieren der Daten und Laden in einen Data Frame
meta_df = json_normalize(dataset_meta)

# Ausgabe
#pd.concat([meta_df], axis=1, sort = False, keys = ["Meta Datensatz: Sports and Outdoors"])

Status der geladenen Zeilen:   0%|          | 0/962300 [00:00<?, ?it/s]

## 4. Allgemeine Bereinigungen

### 4.1. Selectieren der Spalten und Merge

In [ ]:
# Erstellen eines neuen DataFrames mit den relevanten Spalten
review_df = review_df[["reviewText", "asin", "overall", "unixReviewTime", "reviewerID"]]
meta_df = meta_df[["asin", ]]

In [ ]:
# Merge der Data Frames
# Merge der Meta Daten mit dem bereinigeten Datensatz
merged_df = review_df.merge(meta_df, on="asin", how='left')

### 4.2. Nan-Values und doppelte Werte

In [4]:
# Droppen der Nan - Values der reviewTexte
review_df= review_df.dropna(subset=['reviewText'])
review_df.reset_index(inplace=True, drop=False)
review_df= review_df.drop("index", axis=1)

# Entfernung von Dublikaten
review_df.drop_duplicates(inplace=True)
review_df.reset_index(inplace=True, drop=False)
review_df= review_df.drop("index", axis=1)

# Ausgabe
review_df

,reviewText,asin,overall,unixReviewTime,reviewerID
0,What a spectacular tutu! Very slimming.,0000032034,5.0,1433289600,A180LQZBUWVOLF
1,What the heck? Is this a tutu for nuns? I know...,0000032034,1.0,1427846400,ATMFGKU5SVEYY
2,Exactly what we were looking for!,0000032034,5.0,1421107200,A1QE70QBJ8U6ZG
3,I used this skirt for a Halloween costume and ...,0000032034,5.0,1419292800,A22CP6Z73MZTYU
4,This is thick enough that you can't see throug...,0000032034,4.0,1418601600,A22L28G8NRNLLN
...,...,...,...,...,...
2688126,"Love the shorts.. amazing comfortable, perfect...",B01HJGAJ9O,5.0,1522886400,A3TN0U64HONOPB
2688127,Superb quality,B01HJGAJ9O,5.0,1519862400,AJDQLM8PT3YWT
2688128,Works every time,B01HJHHBHG,5.0,1521244800,A3QK5ZLRE2KHLL
2688129,I have a briley bolt release paddle installed ...,B01HJHHBHG,5.0,1509148800,A3VDML80KNR9QQ


### 4.3. Kurze Reviews löschen (<5 Zeichen)

In [6]:
# Funktion zur bestimmung der Länge der Texte pro "Dokument" / Zeile
def count_text_len(in_string):
    text_len = len(in_string.split())
    return text_len

In [7]:
# Anwendung der Funktion zur Ermittlung der Länge (Wörter) der Reviews
review_df["text_len"] = Parallel(n_jobs=-1)(delayed(count_text_len)(in_string) for in_string in tqdm(review_df["reviewText"], 
                                                                                                desc ="Status ermittelten Sprache pro Dokument: ",
                                                                                                total = len(review_df["reviewText"])))

Status ermittelten Sprache pro Dokument:   0%|          | 0/2688640 [00:00<?, ?it/s]

Status ermittelten Sprache pro Dokument: 100%|██████████| 2688640/2688640 [00:17<00:00, 154152.97it/s]


In [10]:
# Löschen der zu kurzen Reviews
review_df = review_df[(review_df["text_len"] >= 5)]

397516


### 4.4. Selectieren der Jahre 2012 bis 2018

In [ ]:
# Umrechnung der unixReviewTime in einen Timestamp
# Quelle: https://www.adamsmith.haus/python/answers/how-to-convert-a-column-of-unix-times-in-a-pandas-dataframe-to-datetimes-in-python
review_df["unixReviewTime"] = pd.to_datetime(review_df["unixReviewTime"], unit='s')

# Aufsplitten in Jahre und Monate
review_df['year'] = pd.DatetimeIndex(review_df["unixReviewTime"]).year

### 4.5. Entfernen der nicht englischen Reviews

In [12]:
# Funktion zur Ermittlung der Sprache
# Quelle: https://pypi.org/project/LanguageIdentifier/
def identifier(in_string):
    language = predict(in_string)
    return language

In [17]:
# Anwenden der Funktion auf den Datensatz
# Durchführen der Sprachanalyse mit Beschleunigung
review_df["language"] = Parallel(n_jobs=-1)(delayed(identifier)(in_string) for in_string in tqdm(review_df["reviewText"], 
                                                                                                desc ="Status ermittelten Sprache pro Dokument: ",
                                                                                                total = len(review_df["reviewText"])))

In [ ]:
# Entfernen der nicht englischen Reviews
review_df = review_df[(review_df["language"] != "en")]

### 4.6. ...

In [28]:
# Erstellen eines neuen DataFrames mit den relevanten Spalten
review_df = review_df[["asin"]]

## 5. Bereinigung der Textdaten

Die Topic Modelling Modelle können lediglich numerische Daten verarbeiten. Aus diesem Grund wird im Rahmen der Textvorverarbeitung die Textdaten in ein berechenbares Zahlenformat konvertiert.
Hierfür existieren eine Reihe von Techniken zur Datenbereinigung und Vorverarbeitung:

- Lowercasing
- Lemmatisierung
- Stemming
- Entfernung von Satzzeichen
- Entfernung von Stopwords
- Entfernung von Zahlen
- Entfernung von nicht ASCII konformen Wörtern
- Entfernung von http und Weblinks

(Pomer, 2022)

Um einen Konsistenten Datensatz für die weitere Verarbeitung und die Anwendung der Topic Modelling Modelle zu erhalten, werden die oben aufgeführten Bereinigungen durchgeführt.

Doch befor die Textbereinigung stattfindet, werden alle Zeilen, welche in den reviewText keine Werte enthalten gelöscht und alle Zeilen rausgenommen, welche keine englischen Reviews enthalten. 
Darüber hinaus werden die Spalten des Data Frame reduziert, indem nur die Spalten "reviewText" und "year" in einen neuen Data Frame aufgenommen werden. Dies hat den Grund, dass lediglich diese Spalten für die Modellierung benötigt werden und die Performance durch die verringerung der Daten verbessert werden kann.

In [ ]:
# Neuer Data Frame, welcher die relevanten Spalten in englischer Sprache beinhaltet
text_df = test_df
text_df["reviewText"] = text_df["reviewText"].astype(str)

### 5.1. Lowercasing

Mithilfe des Lowercasings werden alle Großbuchstaben in Kleinbuchstaben konvertiert, da diese für das Verständnis der Texte nicht benötigt werden. Hierfür wird zuerst eine Funktion definiert, welche einen string in Kleinbuchstaben konvertiert (Pomer, 2022).

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert.
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis des Lowercasing lässt sich in der Ausgabe nachverfolgen.

In [ ]:
# Definition der Funktion, um alle Buchstaben in Kleinschreibung zu konvertieren
# Quelle: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/
def to_lower(in_string):
    out_string = in_string.lower()
    return out_string

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_lower = np.vectorize(to_lower)

# Vektorisierung der Spalte
text_vec = text_df["reviewText"].to_numpy().reshape(-1)

# Durchführen der Sprachanalyse mit Beschleunigung
text_df["text_lower"] = Parallel(n_jobs=-1)(delayed(vfunc_lower)(in_string) for in_string in text_vec)

# Konvertieren der numpy.ndarrays zu einem string
text_df['text_lower'] = text_df['text_lower'].apply(str)

# Ausgabe
pd.concat([text_df[["reviewText", "text_lower"]]], axis=1, sort = False, keys = ["Lowercasing"])

### 5.2. Lemmatisierung
Bei der Lemmatisierung handelt es sich um die Umwandlung von Wörtern in ihre Grundform unter Verwendung von Wörterbüchern (Pomer, 2022). Hierbei werden Pluralformen in Singularformen und unterschiedliche Zeitformen in das Präsenz umgewandelt (Yuzhe-Lu, 2018). Für die Lemmatisierung wird eine Funktion definiert, welche die Sätze in einzelen Token aufsplittet, desse POS-Tags ermittelt und basierend dieser Tags die Lemmatisierung mithilfe der Funktion "lemmatize()" durchführt (Johnson, 2023).  

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert.
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis der Lemmatisierung lässt sich in der Ausgabe nachverfolgen.

In [ ]:
# Definition der Funktion zur Lemmatisierung, um Wörter in ihre Grundform zu konvertieren
# Quelle 1: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle 2: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#stanfordcorenlplemmatization

def lemmatize(in_string):
    # Definition der notwendigen Parameter
    list_pos = 0                                  # Zuordnung einer Positionsnummer
    cleaned_str = ''                              # Leerer String für bereinigte Wörter
    text_token = nltk.word_tokenize(in_string)    # Tokenisieren der Sätze in einzelne Strings
    tagged_words = pos_tag(text_token)            # Grammatikalisches Tagging
    wnl = WordNetLemmatizer()                     # Klasse von NLTK für Lemmatisierung
    
    # Durchfürhung der Lemmatisation und Zusammenführung der Ergebnisse in einen String
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = wnl.lemmatize(word[0], pos='v')
        else:
            lemma = wnl.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_lem = np.vectorize(lemmatize)

# Vektorisierung der Spalte
text_vec = text_df["text_lower"].to_numpy().reshape(-1)

# Durchführen der Sprachanalyse mit Beschleunigung
text_df["text_lemmatize"] = Parallel(n_jobs=-1)(delayed(vfunc_lem)(in_string) for in_string in text_vec)

# Konvertieren der numpy.ndarrays zu einem string
text_df['text_lemmatize'] = text_df['text_lemmatize'].apply(str)

# Ausgabe
pd.concat([text_df[["text_lower", "text_lemmatize"]]], axis=1, sort = False, keys = ["Lemmatizierung"])

### 5.3. Stemming

Da die Lemmatisierung nicht alle Worte bereinigt hat, wie es zum Beispiel bei dem Wort "quickly" zu erkennen ist, welches eigentlich in "quick" konvertiert werden sollte, wird im folgenden zusätzlich der sogenannte Snowball-Stemming-Algorithmus implementiert.

Im Gegensatz zur Lemmatisierung, beruft sich das Stemming nicht auf Wörterbücher, sondern entfernt die Sufixe und Präfixe der Wörter. Es findet daher kein Bezug auf den Kontext der Wörter statt
(Luber & Litzel, Was ist Stemming?, 2020).

Zur Durchführung des Stemming wird eine Funktion definiert, welche die Sätze zuerst in einzelne Token unterteilt und den Snowball Stemmer auf diese mithilfe einer For-Schleife anwendet und wieder zu einem Satz verbindet (NLTK Project, 2023).
Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert. Darüber hinaus werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis des Stemmings lässt sich in der Ausgabe nachverfolgen.

In [ ]:
# Definition der Funktion zur Durchführung des Stemmings
# Quelle: https://www.nltk.org/howto/stem.html
def stemming(in_string):
    # Definition der notwendigen Parameter
    text_token = nltk.word_tokenize(in_string)      # Tokenisieren der Sätze in einzelne Strings
    stemmer = SnowballStemmer("english")            # Laden der Klasse zum Stemming
    
    # Durchfürhung des Stemmings und Zusammenführung der Ergebnisse in einen String
    cleaned_str = ' '.join([stemmer.stem(word) for word in text_token])
    return cleaned_str

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_stem = np.vectorize(stemming)

# Vektorisierung der Spalte
text_vec = text_df["text_lemmatize"].to_numpy().reshape(-1)

# Durchführen der Sprachanalyse mit Beschleunigung
text_df["text_stemming"] = Parallel(n_jobs=-1)(delayed(vfunc_stem)(in_string) for in_string in text_vec)

# Konvertieren der numpy.ndarrays zu einem string
text_df['text_stemming'] = text_df['text_stemming'].apply(str)

# Ausgabe
pd.concat([text_df[["text_lemmatize", "text_stemming"]]], axis=1, sort = False, keys = ["Stemming"])

### 5.4. Entfernung von Satzzeichen

Nachdem die bereinigung der Wörter selbst abgeschlossen ist, werden nun die Satzzeichen aus dem Datensatz entfernt. Viele Satzzeichen verändern bei ihrem Fehlen den Kontext nicht groß, sodass diese problemlos bereinigt werden können. Jedoch ist zu beachten, dass beispielsweise Kommata oder Fragezeichen den Inhalt eines Textes verändern können, sodass es bei der Analyse zu Fehlinterpretationen kommt. Besonders zu beachten sind in diesem Zusammenhang Zahlen, da es bei dem Jahr "1998" und einem Geldwert "19,98" einen großen Unterschied macht, ob das Komma im Datensatz erhalten bleibt oder ob es verschwindet. Bei letzteren Fall hätten beide Zahlen den identischen Wert und das Ergebnis der Auswertung hätte keine Aussagekraft (Pomer, 2022). Aus diesem Grund soll im Folgenden mit diesem Wissen im Hinterkopf eine Bereinigung der Satzzeichen stattfinden. Treten bei dem Ergebnis der Analyse vermehrt Zahlen oder andere Unstimmigkeiten auf, müssen die Satzzeichen ggf. in die Modelle miteinbezogen werden.

Für die Bereinigung werden folgende Zeichen durch einen leeren Wert mithilfe der PySpark functions "col()" und translate()" ersetzt. Der Import der functions ist in Abschnitt 1. Installationen und Imports zu finden (Apache Spark, kein Datum):

> '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'


In [ ]:
# Funktion zur Entfernung der Punktierung
# Quelle: https://www.geeksforgeeks.org/python-remove-punctuation-from-string/
def punctation(in_string):
    cleaned_string = in_string.translate(str.maketrans("","",string.punctuation))
    return cleaned_string

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_punct = np.vectorize(punctation)

# Vektorisierung der Spalte
text_vec = text_df["text_stemming"].to_numpy().reshape(-1)

# Durchführen der Sprachanalyse mit Beschleunigung
text_df["text_punctation"] = Parallel(n_jobs=-1)(delayed(vfunc_punct)(in_string) for in_string in text_vec)

# Konvertieren der numpy.ndarrays zu einem string
text_df['text_punctation'] = text_df['text_punctation'].apply(str)

# Ausgabe
pd.concat([text_df[["text_stemming", "text_punctation"]]], axis=1, sort = False, keys = ["Punctation"])

### 5.5. Entfernung von Stopwords

Stoppwörter können in Vorbereitung auf das Topic Modelling ebenfalls herausgefiltert werden, da diese keinen Einfluss auf die Bedeutung des Kontextes haben. Aufgrund ihrer Häufigkeit in einem Text wird darüber hinaus der Datensatz drastisch reduziert, wordurch die Laufzeit sowie die Genauigkeit der Analyse verbessert werden kann. Die Auswahl der Stopword muss jedoch bedacht getroffen werden, um die Ergebnisse nicht zu verfälschen. So ist beispielsweise die Negation "nicht" in vielen Sprachen als Stopword eingestuft, kann jedoch den Kontext eines Satzes bei dessen Entfernung dramatisch verändern (Teja, 2020). Für jede Sprache existieren unterschiedliche Stoppwörter. Um diese aus den Daten herauszufiltern werden sogenannte Stoppwordlisten herangezogen (Pomer, 2022).

Für die Bereinigung der Daten im Rahmen dieser Masterarbeit wird eine Liste der englischen Stopword der NLTK Library verwendet. 
Zur Durchführung der Bereinigung wird eine Funktion definiert, welche die Token des Amazon Datensatzes mit der Liste der Stopwords vergleicht und einen bereinigten String kreiert, welcher keine Stopwords mehr enthält (Yuzhe-Lu, 2018).

In [ ]:
# Definition der Funktion zur Entfernung der Stopwords
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_stops(in_string):
    # Definition der notwendigen Parameter
    list_pos = 0                                  # Zuordnung einer Positionsnummer
    cleaned_str = ''                              # Leerer String für bereinigte Wörter
    text_token = nltk.word_tokenize(in_string)    # Tokenisieren der Sätze in einzelne Strings
    stop_words = stopwords.words('english')       # Bestimmen der Stopwords (für Englisch)
    stop_words.append('would')                    # Hinzufügen individueller Wörter zur Liste der Stopwords

    # Durchfürhung der Entfernung der Stopwords und Zusammenführung der Ergebnisse in einen String
    for word in text_token:
        if word not in stop_words:
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_stop = np.vectorize(remove_stops)

# Vektorisierung der Spalte
test_df["text_stopwords"] = vfunc_stop(text_df["text_punctation"].to_numpy().reshape(-1))

# Ausgabe
pd.concat([text_df[["text_punctation", "text_stopwords"]]], axis=1, sort = False, keys = ["Stopwords"])

### 5.6. Entfernung von Zahlen

Im Rahmen der Masterarbeit wurde beschlossen, die Zahlen aus dem Datensatz zu entfernen, da die Annahme getroffen wurde, dass diese nicht zur Einteilung in die richtigen Kategorien beitragen.
Mithilfe der Regular expressions werden die Nummern 0 bis 9 in einerm Parameter gespeichert. Dieser wird daraufhin mit dem Input Text abgeglichen und bei einer Übereinstimmung wird die Zahl mit "nichts" ausgetauscht (Python Software Foundation, 2023). 

Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert (Johnson, 2023).
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [ ]:
# Definition der Funktion zur Entfernung der Zahlen
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
# Quelle: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/
def remove_numbers(in_string):
    # Kompilieren der Zahlen 0-9 in ein Ausdrucksmuster
    num_re = re.compile('(\\d+)')

    # Ersetzen der Zahlen durch "nichts"
    cleaned_str = " ".join((re.sub(num_re, "", in_string)).split()) 
    return cleaned_str

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_numbers = np.vectorize(remove_numbers)

# Vektorisierung der Spalte
text_vec = text_df["text_stopwords"].to_numpy().reshape(-1)

# Durchführen der Sprachanalyse mit Beschleunigung
text_df["text_numbers"] = Parallel(n_jobs=-1)(delayed(vfunc_numbers)(in_string) for in_string in text_vec)

# Konvertieren der numpy.ndarrays zu einem string
text_df['text_numbers'] = text_df['text_numbers'].apply(str)

# Ausgabe
pd.concat([text_df[["text_stopwords", "text_numbers"]]], axis=1, sort = False, keys = ["Numbers"])

### 5.7. Entfernung von nicht ASCII konvormen Wörtern

ASCII ist die Kurzform für „American Standard Code for Information Interchange“ und dient zur standartisierten Darstellung von Zeichen in elektronischer Form (IONOS SE, 2022). Zu beachten ist, dass es bei diesem um einen amerikanischen Standart handelt und daher keine Sonderzeichen, wie „ß“ oder „é, á“ berücksichtigt und daher beispielsweise bei deutschen Datensätzen das ergebnis verfälschen könnte (Pomer, 2022). Da der Amazondatensatz zufor auf die Sprache geprüft und bereinigt wurde, beinhaltet dieser lediglich englische Sätze, wodurch der ASCII Standard verwendet werden kann.

Zur Bereinigung wird die untenstehende Funktion definiert, welche den Unicode der wörter ermittelt und diese mit den Unicodes aus dem ASCII Standard abgleicht. Lediglich die Wörter, welche zwischen 0 und 128 liegen werden im Datensatz beibehalten.
Im Anschluss wird mithilfe von PySpark die Funktion auf den Data Frame angewendet und die bereinigten Ergebnisse in einer neuen Spalte gespeichert (Johnson, 2023).
Abschließend werden alle "non-type-values" aus dem Datensatz herausgefilter, bevor dieser den nächsten Bereinigungsschritt durchläuft (Yuzhe-Lu, 2018).
Das Ergebnis für die Entfernung der Satzzeichen lässt sich in der Ausgabe nachverfolgen.

In [ ]:
# Definition der Funktion zur Entfernung der ASCII characters
# Quelle: https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb
def remove_non_ascii_words(in_string):
    ''' Returns the string without non ASCII characters'''
    cleaned_str = ''.join(word for word in in_string if 0 < ord(word) < 128)
    return cleaned_str

In [ ]:
# Anwenden der Funktion auf den Datensatz
# Erstellung einer Vektor Funktion
vfunc_ascii = np.vectorize(remove_non_ascii_words)

# Vektorisierung der Spalte
text_vec = text_df["text_numbers"].to_numpy().reshape(-1)

# Durchführen der Sprachanalyse mit Beschleunigung
text_df["text_ascII"] = Parallel(n_jobs=-1)(delayed(vfunc_ascii)(in_string) for in_string in text_vec)

# Konvertieren der numpy.ndarrays zu einem string
text_df['text_ascII'] = text_df['text_ascII'].apply(str)

# Ausgabe
pd.concat([text_df[["text_numbers", "text_ascII"]]], axis=1, sort = False, keys = ["ASCII Characters"])

### 5.9. Merge und select


In [ ]:
# Merge der Meta Daten mit dem bereinigeten Datensatz
merged_df = text_df.merge(meta_df, on="asin", how='left')

In [ ]:
# Neuen Data Frame erzeugen, welcher nur die relevanten Spalten enhält
cleaned_data_df = merged_df[["reviewText", "text_ascII", "year", "month", "overall", "brand", "price", "title"]]

In [ ]:
cleaned_data_df.head()

# Export

Apache Spark. (17. 02 2023). Download Apache Spark™. Abgerufen am 20. 02 2023 von spark.apache.org: https://spark.apache.org/downloads.html

Erick. (12. 02 2018). How to unzip gz file using Python. Abgerufen am 24. 10 2022 von stackoverflow.com: https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python

Free Software Foundation, I. (02. 04 2022). GNU Gzip: General file (de)compression. Abgerufen am 20. 02 2023 von gnu.org: https://www.gnu.org/software/gzip/manual/gzip.html

IONOS SE. (05. 04 2022). ASCII – Erklärung und Beispiele. Abgerufen am 08. 03 2023 von ionos.de: https://www.ionos.de/digitalguide/server/knowhow/ascii-american-standard-code-for-information-interchange/

Johnson, D. (21. 01 2023). POS-Tagging mit NLTK und Chunking in NLP [BEISPIELE]. Abgerufen am 21. 02 2023 von guru99.com: https://www.guru99.com/pos-tagging-chunking-nltk.html#1

Luber, D.-I. & Litzel, N. (30. 11 2020). Was ist Stemming? Abgerufen am 22. 02 2023 von bigdata-insider.de: https://www.bigdata-insider.de/was-ist-stemming-a-980852/#:~:text=Abgrenzung%20von%20Stemming%20und%20Lemmatisierung&text=Stemming%2DAlgorithmen%20arbeiten%20meist%20mit,)%2C%20um%20Stammformen%20zu%20finden.

NLTK Project. (02. 01 2023). Natural Language Toolkit - Sample usage for stem. Abgerufen am 22. 02 2023 von nltk.org: https://www.nltk.org/howto/stem.html

Oracle. (17. 01 2023). Download Java for Windows. Abgerufen am 20. 02 2023 von java.com: https://www.java.com/download/ie_manual.jsp

Pomer, L. (03. 08 2022). Textklassifikation – Vorverarbeitung der Daten. Abgerufen am 14. 02 2023 von thecattlecrew.net: https://thecattlecrew.net/2022/08/03/textklassifikation-vorverarbeitung-der-daten/

Preusler, S. (13. 03 2020). Pyspark und Jupyter Notebook Anleitung für Windows. Abgerufen am 14. 01 2023 von https://medium.com/@stefan.preusler/pyspark-und-jupyter-notebook-anleitung-f%C3%BCr-windows-7317f2c968c4

Python Software Foundation. (2023). re — Regular expression operations¶. Abgerufen am 08. 03 2023 von docs.python.org: https://docs.python.org/3/library/re.html

Shah, A., 2018. How to setup Apache Spark(PySpark) on Jupyter/IPython Notebook?. [Online] 
Available at: https://medium.com/@ashish1512/how-to-setup-apache-spark-pyspark-on-jupyter-ipython-notebook-3330543ab307
[Accessed 14 01 2023].

Sarkar, T. (12. 11 2018). How to set up PySpark for your Jupyter notebook. Abgerufen am 14. 01 2023 von opensource.com: https://opensource.com/article/18/11/pyspark-jupyter-notebook

SparkBy{Examples}. (06. 12 2020). PySpark Liest JSON-Datei in DataFrame. Abgerufen am 14. 01 2023 von https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/#read-json-multiline

Teja, S. (10. 06 2020). Stop Words in NLP. Abgerufen am 24. 02 2023 von medium.com: https://medium.com/@saitejaponugoti/stop-words-in-nlp-5b248dadad47

Wuttke, L. (2022). Einführung in Apache Spark: Komponenten, Vorteile und Anwendungsbereiche. Abgerufen am 05. 01 2023 von datasolut.com: https://datasolut.com/was-ist-spark/#was-ist-pyspark

Yuzhe-Lu. (10. 12 2018). Topic-Modeling-on-Amazon-Reviews-using-LDA/2_LDA_Data_Processing.ipynb. Abgerufen am 14. 02 2023 von https://github.com/rrathgithub/Topic-Modeling-on-Amazon-Reviews-using-LDA/blob/master/2_LDA_Data_Processing.ipynb